In [1]:
import os
import pandas as pd
import lightgbm as lgb

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:

# Function to load and preprocess data from CSV files
def load_and_preprocess_data(directory):
    feature_list = []
    ranking_list = []
    for file_name in os.listdir(directory):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory, file_name)
            data = pd.read_csv(file_path)
            features = data[['#candidates', 'type', 'level']].copy()
            features['num_voters'] = len(data)
            rankings = data['rankings']
            feature_list.append(features)
            ranking_list.append(rankings)
    # Combine all features and rankings into a single DataFrame
    combined_features = pd.concat(feature_list, ignore_index=True)
    combined_rankings = pd.concat(ranking_list, ignore_index=True)
    return combined_features, combined_rankings

# Function to train a ranking model
def train_ranking_model(features, rankings):
    train_data = lgb.Dataset(features, label=rankings)
    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'ndcg_at': [5],  # Adjust based on your requirement
        'learning_rate': 0.1,
        'num_leaves': 31,
        'min_data_in_leaf': 20,
    }
    ranker = lgb.train(params, train_data, num_boost_round=100)
    return ranker

# Function to predict rankings for new data
def predict_rankings(model, features):
    predictions = model.predict(features)
    ranked_items = [sorted(range(len(p)), key=lambda i: p[i], reverse=True) for p in predictions]
    return ranked_items

# Function to save the predicted rankings to a new CSV file
def save_predictions(predictions, output_file_path):
    predictions_df = pd.DataFrame(predictions)
    predictions_df.to_csv(output_file_path, index=False)

# Main function to process all files in a directory and generate predictions
def main(input_directory, output_file_path):
    # Load and preprocess training data
    features, rankings = load_and_preprocess_data(input_directory)
    
    # Train the ranking model
    ranker = train_ranking_model(features, rankings)
    
    # Predict rankings for the features
    predictions = predict_rankings(ranker, features)
    
    # Save predictions to output file
    save_predictions(predictions, output_file_path)

# Directory containing the input CSV files and the output file path
input_directory = 'path/to/csv_files'
output_file_path = 'predicted_rankings.csv'

# Run the main function
main(input_directory, output_file_path)


NULL Elections


In [1]:
from null_elections import *

In [3]:
filename = "Maine_11062018_CongressionalDistrict2.csv"
runs = 1
sum_gamma = 0
for i in range(runs):
    gamma = get_null_gamma(filename)[1]
    sum_gamma += gamma

print(sum_gamma/runs)

0.7881019751920565


In [3]:

filename = "Maine_11082022_CongressionalDistrict2.csv"
runs = 5
sum_gamma = 0
for i in range(runs):
    gamma = get_null_gamma(filename)[1]
    sum_gamma += gamma

print(sum_gamma/runs)

KeyboardInterrupt: 

In [3]:
directory = "null_elections"
csv = os.path.join(directory, filename) 
df = pd.read_csv(csv)
candidates = df['candidate'].values
print(candidates)

['REP Poliquin, Bruce' 'DEM Golden, Jared F.' 'Bond, Tiffany L.'
 'Hoar, William R.S.']


In [28]:
import sys
!{sys.executable} -m pip install python-constraint

  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24081 sha256=2cdd4dc30c8fd4c63336be621649299cd1924a9abcaa85b201d86051b245b4ae
  Stored in directory: c:\users\mahsh\appdata\local\pip\cache\wheels\51\36\1f\c2ccb8dc4eba38c5215636d4ae2c480b32069cab0376bcc1a4
Successfully built python-constraint


In [30]:
from constraint import Problem, AllDifferentConstraint

def find_all_solutions_cp(Va, Vb, Vc, len_1, len_2, len_3, max_value):
    problem = Problem()

    # Add variables with their respective domains
    problem.addVariables(['a1', 'a2', 'a3', 'a4', 'a5'], range(max_value + 1))
    problem.addVariables(['b1', 'b2', 'b3', 'b4', 'b5'], range(max_value + 1))
    problem.addVariables(['c1', 'c2', 'c3', 'c4', 'c5'], range(max_value + 1))

    # Add constraints
    problem.addConstraint(lambda a1, a2, a3, a4, a5: a1 + a2 + a3 + a4 + a5 == Va, ['a1', 'a2', 'a3', 'a4', 'a5'])
    problem.addConstraint(lambda b1, b2, b3, b4, b5: b1 + b2 + b3 + b4 + b5 == Vb, ['b1', 'b2', 'b3', 'b4', 'b5'])
    problem.addConstraint(lambda c1, c2, c3, c4, c5: c1 + c2 + c3 + c4 + c5 == Vc, ['c1', 'c2', 'c3', 'c4', 'c5'])

    problem.addConstraint(lambda a1, b1, c1: a1 + b1 + c1 == len_1, ['a1', 'b1', 'c1'])
    problem.addConstraint(lambda a2, a3, b2, b3, c2, c3: a2 + a3 + b2 + b3 + c2 + c3 == len_2, ['a2', 'a3', 'b2', 'b3', 'c2', 'c3'])
    problem.addConstraint(lambda a4, a5, b4, b5, c4, c5: a4 + a5 + b4 + b5 + c4 + c5 == len_3, ['a4', 'a5', 'b4', 'b5', 'c4', 'c5'])

    # Find all solutions
    solutions = problem.getSolutions()

    return solutions

# Example usage with arbitrary values and a maximum value bound
Va = 9
Vb = 7
Vc = 3
len_1 = 5
len_2 = 8
len_3 = 6
max_value = 10

solutions = find_all_solutions_cp(Va, Vb, Vc, len_1, len_2, len_3, max_value)
print(f"Found {len(solutions)} solutions.")
for sol in solutions:
    print(sol)


KeyboardInterrupt: 

In [21]:
import pulp
import random

# Example parameters (replace with your actual values)
Va = 9
Vb = 7
Vc = 3
len_1 = 5
len_2 = 8
len_3 = 6


# Initialize the LP problem
prob = pulp.LpProblem("RandomizedIntegerSolution", pulp.LpMaximize)

# Define variables
a = pulp.LpVariable.dicts('a', range(1, 6), lowBound=0, cat='Integer')
b = pulp.LpVariable.dicts('b', range(1, 6), lowBound=0, cat='Integer')
c = pulp.LpVariable.dicts('c', range(1, 6), lowBound=0, cat='Integer')

# Objective function (not really needed for feasibility check)
prob += 0

# Constraints
prob += pulp.lpSum(a[i] for i in range(1, 6)) == Va
prob += pulp.lpSum(b[i] for i in range(1, 6)) == Vb
prob += pulp.lpSum(c[i] for i in range(1, 6)) == Vc

prob += a[1] + b[1] + c[1] == len_1
prob += a[2] + a[3] + b[2] + b[3] + c[2] + c[3] == len_2
prob += a[4] + a[5] + b[4] + b[5] + c[4] + c[5] == len_3

# Number of randomized runs to attempt
num_runs = 10

# List to store all found solutions
all_solutions = []

# Solve the problem multiple times with random initial values
for _ in range(num_runs):
    # Randomly initialize variables
    random.seed()  # Use current system time as seed for randomness
    for i in range(1, 6):
        a[i].setInitialValue(random.randint(0, Va))
        b[i].setInitialValue(random.randint(0, Vb))
        c[i].setInitialValue(random.randint(0, Vc))

    # Solve the problem
    prob.solve()

    # Store the solution if optimal
    if pulp.LpStatus[prob.status] == "Optimal":
        solution = [(int(a[i].value()), int(b[i].value()), int(c[i].value())) for i in range(1, 6)]
        all_solutions.append(solution)

# Print all found solutions
if all_solutions:
    for idx, solution in enumerate(all_solutions):
        print(f"Solution {idx + 1}:")
        for i in range(5):
            print(f"a[{i + 1}] = {solution[i][0]}, b[{i + 1}] = {solution[i][1]}, c[{i + 1}] = {solution[i][2]}")
        print()
else:
    print("No optimal solutions found in the given number of runs.")


Solution 1:
a[1] = 1, b[1] = 1, c[1] = 3
a[2] = 8, b[2] = 0, c[2] = 0
a[3] = 0, b[3] = 0, c[3] = 0
a[4] = 0, b[4] = 6, c[4] = 0
a[5] = 0, b[5] = 0, c[5] = 0

Solution 2:
a[1] = 1, b[1] = 1, c[1] = 3
a[2] = 8, b[2] = 0, c[2] = 0
a[3] = 0, b[3] = 0, c[3] = 0
a[4] = 0, b[4] = 6, c[4] = 0
a[5] = 0, b[5] = 0, c[5] = 0

Solution 3:
a[1] = 1, b[1] = 1, c[1] = 3
a[2] = 8, b[2] = 0, c[2] = 0
a[3] = 0, b[3] = 0, c[3] = 0
a[4] = 0, b[4] = 6, c[4] = 0
a[5] = 0, b[5] = 0, c[5] = 0

Solution 4:
a[1] = 1, b[1] = 1, c[1] = 3
a[2] = 8, b[2] = 0, c[2] = 0
a[3] = 0, b[3] = 0, c[3] = 0
a[4] = 0, b[4] = 6, c[4] = 0
a[5] = 0, b[5] = 0, c[5] = 0

Solution 5:
a[1] = 1, b[1] = 1, c[1] = 3
a[2] = 8, b[2] = 0, c[2] = 0
a[3] = 0, b[3] = 0, c[3] = 0
a[4] = 0, b[4] = 6, c[4] = 0
a[5] = 0, b[5] = 0, c[5] = 0

Solution 6:
a[1] = 1, b[1] = 1, c[1] = 3
a[2] = 8, b[2] = 0, c[2] = 0
a[3] = 0, b[3] = 0, c[3] = 0
a[4] = 0, b[4] = 6, c[4] = 0
a[5] = 0, b[5] = 0, c[5] = 0

Solution 7:
a[1] = 1, b[1] = 1, c[1] = 3
a[2] = 8, b